In [2]:
import networkx
# https://ithelp.ithome.com.tw/articles/10218768 conda ver.


In [ ]:
# it's non-matrices


In [3]:
import numpy as np


def find_min_time(graph, cost_tables, start, end, max_cost):
    # Initialize DP and path tables
    num_vertices = len(graph)
    dp = [[float('inf')] * (max_cost + 1) for _ in range(num_vertices)]
    path = [[[] for _ in range(max_cost + 1)] for _ in range(num_vertices)]

    # Initialize starting vertex
    dp[start][0] = 0

    # Iterate through vertices
    for v in range(num_vertices):
        for cost in range(max_cost + 1):
            if dp[v][cost] == float('inf'): # Skip if vertex v is not reachable
                continue

            # Iterate through neighbors of vertex v
            for u in range(num_vertices):
                if v != u and graph[v][u] == 1:
                    for edge_cost, edge_time in cost_tables[v][u]:
                        new_cost = cost + edge_cost                 # CHECK density? vs face
                        new_time = dp[v][cost] + edge_time

                        # Check if the new cost is within budget
                        if new_cost <= max_cost and new_time < dp[u][new_cost]:
                            dp[u][new_cost] = new_time
                            path[u][new_cost] = path[v][cost] + [(v, u)]

    # Find the minimum time within budget
    min_time = min(dp[end])

    # Find the path corresponding to the minimum time
    min_time_path = path[end][dp[end].index(min_time)]

    return min_time, min_time_path


In [4]:
# Example usage
graph = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
cost_table_AB = [(100, 10), (60, 20), (40, 40)]
cost_table_BC = [(40, 10), (30, 15), (20, 20)]
cost_table_AC = [(30, 80), (20, 90), (10, 100)]

cost_tables = [[[], cost_table_AB, cost_table_AC],
               [cost_table_AB, [], cost_table_BC],
               [cost_table_AC, cost_table_BC, []]]

start_vertex = 0  # A
end_vertex = 2    # C
max_allowed_cost = 100

min_time, min_time_path = find_min_time(
    graph, cost_tables, start_vertex, end_vertex, max_allowed_cost)

print("Minimum Time:", min_time)
print("Optimal Path:", min_time_path)


Minimum Time: 30
Optimal Path: [(0, 1), (1, 2)]


In [9]:
graph = np.array([[0, 1, 1, 0, 0],
                  [1, 0, 1, 1, 0],
                  [1, 1, 0, 0, 1],
                  [0, 1, 0, 0, 1],
                  [0, 0, 1, 1, 0]])

cost_table_AB = [(100, 10), (60, 20), (40, 40)]
cost_table_AC = [(30, 80), (20, 90), (10, 100)]
cost_table_BC = [(40, 10), (30, 15), (20, 20)]
cost_table_BD = [(60, 10), (50, 20), (30, 40)]
cost_table_CE = [(70, 15), (40, 30), (20, 20)]

cost_tables = [[[], cost_table_AB, cost_table_AC, [], []],
               [cost_table_AB, [], cost_table_BC, cost_table_BD, []],
               [cost_table_AC, cost_table_BC, [], [], cost_table_CE],
               [[], cost_table_BD, [], [], []],
               [[], [], cost_table_CE, [], []]]

start_vertex = 0  # Vertex A
end_vertex = 4    # Vertex E
max_allowed_cost = 150

min_time, min_time_path = find_min_time(
    graph, cost_tables, start_vertex, end_vertex, max_allowed_cost)

print("Minimum Time:", min_time)
print("Optimal Path:", min_time_path)


Minimum Time: 45
Optimal Path: [(0, 1), (1, 2), (2, 4)]


In [ ]:
"""
  Finds all possible arrive times from a start node to an end node in a graph, subject to the constraint that the cost of the path must be less than or equal to a maximum cost.

  Args:
    adjacency_matrix: A square matrix representing the graph, where each element (i, j) is the cost of the edge from vertex i to vertex j, or 0 if there is no edge from vertex i to vertex j.
    cost_tables: A dictionary mapping edge (u, v) to a list of (cost, time) pairs.
    start_vertex: The start node.
    end_vertex: The end node.
    max_cost: The maximum allowed cost.

  Returns:
    A list of arrive times, where each arrive time is represented by a tuple (cost, time, path).
  """


In [5]:
def find_all_arrive_times(adjacency_matrix, cost_tables, start_vertex, end_vertex, max_cost):
  
  # Initialize DP table
  num_vertices = len(adjacency_matrix)
  dp = [[float('inf')] * (max_cost + 1) for _ in range(num_vertices)]
  path = [[[] for _ in range(max_cost + 1)] for _ in range(num_vertices)]

  # Initialize starting vertex
  dp[start_vertex][0] = 0

  # Iterate through vertices
  for v in range(num_vertices):
    for cost in range(max_cost + 1):
      if dp[v][cost] == float('inf'): # Skip if vertex v is not reachable
        continue

      # Iterate through neighbors of vertex v
      for u in range(num_vertices):
        if v != u and adjacency_matrix[v][u] == 1 and v != end_vertex and (v, u) in cost_tables:  # Check if edge exists;  check (v, u) in cost_tables: data may be missing
          for edge_cost, edge_time in cost_tables[(v, u)]:
            new_cost = cost + edge_cost
            new_time = dp[v][cost] + edge_time

            # Check if the new cost is within budget
            if new_cost <= max_cost:
              # Update the DP table and path if the new time is shorter
              if new_time < dp[u][new_cost]:
                dp[u][new_cost] = new_time
                path[u][new_cost] = path[v][cost] + [(v, u)]
              elif new_time == dp[u][new_cost]:
                # If arrival time is the same, choose the shorter path
                if len(path[v][cost]) + 1 < len(path[u][new_cost]):
                  path[u][new_cost] = path[v][cost] + [(v, u)]

  # Get all possible arrive times
  all_arrive_times = []
  for cost in range(max_cost + 1):
    if dp[end_vertex][cost] != float('inf'):
      all_arrive_times.append((cost, dp[end_vertex][cost], path[end_vertex][cost]))

  return all_arrive_times


In [39]:
# Adjacency matrix
adjacency_matrix = [[0, 1, 0, 0, 0],
                   [1, 0, 1, 1, 0],
                   [0, 1, 0, 0, 1],
                   [0, 1, 0, 0, 1],
                   [0, 0, 1, 1, 0]]

# Cost tables
cost_tables = {(0, 1): [(100, 10), (60, 20), (40, 40)],
               (1, 2): [(8, 10), (4, 15), (2, 20)],
               (1, 3): [(50, 30), (40, 45), (30, 70)],
               (3, 4): [(60, 10), (50, 15), (40, 20)],
               (2, 4): [(70, 50), (60, 75), (50, 100)]}

# Start and end nodes
start_vertex = 0
end_vertex = 4

# Maximum allowed cost
max_cost = 120

# Find all possible arrive times
all_arrive_times = find_all_arrive_times(adjacency_matrix, cost_tables, start_vertex, end_vertex, max_cost)

# Print all possible arrive times
for cost, time, path in all_arrive_times:
  print(f"Cost: {cost}, Arrival Time: {time}, Path: {path}")


Cost: 92, Arrival Time: 160, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 94, Arrival Time: 155, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 98, Arrival Time: 150, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 102, Arrival Time: 135, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 104, Arrival Time: 130, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 108, Arrival Time: 125, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 110, Arrival Time: 130, Path: [(0, 1), (1, 3), (3, 4)]
Cost: 112, Arrival Time: 110, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 114, Arrival Time: 105, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 118, Arrival Time: 100, Path: [(0, 1), (1, 2), (2, 4)]
Cost: 120, Arrival Time: 105, Path: [(0, 1), (1, 3), (3, 4)]


In [38]:
# Adjacency matrix
adjacency_matrix = [[0, 1, 1],
                    [1, 0, 1],
                    [1, 1, 0]]

# Cost tables
cost_tables = {(0, 1): [(100, 10), (60, 20), (40, 40)],
               (1, 2): [(40, 10), (30, 15), (20, 20)],
               (0, 2): [(70, 10), (60, 15), (50, 20)]}

# Start and end nodes
start_vertex = 0
end_vertex = 2

# Maximum allowed cost
max_cost = 120

# Find all possible arrive times
all_arrive_times = find_all_arrive_times(adjacency_matrix, cost_tables, start_vertex, end_vertex, max_cost)

# Print all possible arrive times
for cost, time, path in all_arrive_times:
    print(f"Cost: {cost}, Arrival Time: {time}, Path: {path}")


Cost: 50, Arrival Time: 20, Path: [(0, 2)]
Cost: 60, Arrival Time: 15, Path: [(0, 2)]
Cost: 70, Arrival Time: 10, Path: [(0, 2)]
Cost: 80, Arrival Time: 40, Path: [(0, 1), (1, 2)]
Cost: 90, Arrival Time: 35, Path: [(0, 1), (1, 2)]
Cost: 100, Arrival Time: 30, Path: [(0, 1), (1, 2)]
Cost: 120, Arrival Time: 30, Path: [(0, 1), (1, 2)]


In [10]:
import numpy as np

# Define the graph structure (adjacency matrix)
graph = np.array([
    [0, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [1, 0, 0, 0, 1],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0]
])

# Define cost tables for each edge
cost_table_AB = [(100, 10), (60, 20), (40, 40)]
cost_table_AC = [(30, 80), (20, 90), (10, 100)]
cost_table_BC = [(40, 10), (30, 15), (20, 20)]
cost_table_BD = [(60, 10), (50, 20), (30, 40)]
cost_table_CE = [(70, 15), (40, 30), (20, 20)]

# Define parameters
start_vertex = 0
end_vertex = 4
max_cost = 150

# Initialize a table to store time combinations
# table[i][j] will store all possible times to reach vertex j with a cost of i
num_vertices = graph.shape[0]
table = [[[] for _ in range(num_vertices)] for _ in range(max_cost + 1)]

# Initialize the table for the starting vertex
for cost, time in cost_table_AB:
    if cost <= max_cost:
        table[cost][1].append(time)

# Dynamic Programming to fill in the table
for v in range(2, num_vertices):
    for cost in range(1, max_cost + 1):
        for u in range(num_vertices):
            if graph[u][v]:
                for edge_cost, edge_time in cost_table_CE:
                    if cost - edge_cost >= 0:
                        for prev_time in table[cost - edge_cost][u]:
                            table[cost][v].append(prev_time + edge_time)

# Display the table
print("Cost\tVertex\tPossible Times")
for cost in range(1, max_cost + 1):
    for v in range(num_vertices):
        times = table[cost][v]
        if times:
            print(f"{cost}\t{v}\t{sorted(set(times))}")


Cost	Vertex	Possible Times
40	1	[40]
60	1	[20]
60	3	[60]
80	3	[40, 70]
100	1	[10]
100	3	[50]
110	3	[55]
120	3	[30]
130	3	[35]
140	3	[40]


## Face edge weight implement: 
calculate edge weights for a geometric graph based on a density function 
1. KDE
2. k-NN
3. ϵ-graph


In [ ]:
#KDE function:
def fp_linear(x, m, b):
    return m * x + b
def calculate_wij(xi, xj, ϵ):
    distance = np.linalg.norm(xi - xj)
    if distance <= ϵ:
        return fp_linear((xi + xj) / 2, m, b) * distance
    else:
        return 0
m, b = 2, 1
ϵ = 1
N = 100

result_wij = calculate_wij(xi, xj, ϵ)

In [1]:
import numpy as np

def find_all_arrive_times(adjacency_matrix, cost_tables, start_vertex, end_vertex, max_cost, data, bandwidth):
    num_vertices = len(adjacency_matrix)
    dp = [[float('inf')] * (max_cost + 1) for _ in range(num_vertices)]
    path = [[[] for _ in range(max_cost + 1)] for _ in range(num_vertices)]

    # Initialize starting vertex
    dp[start_vertex][0] = 0

    # KDE-based edge weight calculation
    num_samples, num_features = data.shape
    edge_weights = np.zeros((num_samples, num_samples))
    
    for i in range(num_samples):
        for j in range(i + 1, num_samples):
            mid_point = (data[i] + data[j]) / 2
            density_estimate = np.sum(np.exp(-0.5 * ((np.linalg.norm(data[i] - data[j]) / bandwidth) ** 2)))
            edge_weights[i, j] = density_estimate * np.linalg.norm(data[i] - data[j])
    
    for v in range(num_vertices):
        for cost in range(max_cost + 1):
            if dp[v][cost] == float('inf'):
                continue
            for u in range(num_vertices):
                if v != u and adjacency_matrix[v][u] == 1 and v != end_vertex and (v, u) in cost_tables:
                    for edge_cost, edge_time in cost_tables[(v, u)]:
                        new_cost = cost + edge_cost
                        new_time = dp[v][cost] + edge_time
                        if new_cost <= max_cost:
                            if new_time < dp[u][new_cost]:
                                dp[u][new_cost] = new_time
                                path[u][new_cost] = path[v][cost] + [(v, u)]
                            elif new_time == dp[u][new_cost]:
                                if len(path[v][cost]) + 1 < len(path[u][new_cost]):
                                    path[u][new_cost] = path[v][cost] + [(v, u)]
    
    all_arrive_times = []
    for cost in range(max_cost + 1):
        if dp[end_vertex][cost] != float('inf'):
            all_arrive_times.append((cost, dp[end_vertex][cost], path[end_vertex][cost]))

    return all_arrive_times



In [3]:
# Example usage
adjacency_matrix = [[0, 1, 1],
                    [1, 0, 1],
                    [1, 1, 0]]
cost_tables = {(0, 1): [(100, 10), (60, 20), (40, 40)],
               (1, 2): [(40, 10), (30, 15), (20, 20)],
               (0, 2): [(70, 10), (60, 15), (50, 20)]}
start_vertex = 0
end_vertex = 2
max_cost = 120

# Sample data for KDE
data = np.array([[0.0, 0.0], [1.0, 1.0], [2.0, 2.0]])

# Bandwidth parameter for KDE
bandwidth = 1.0

all_arrive_times = find_all_arrive_times(adjacency_matrix, cost_tables, start_vertex, end_vertex, max_cost, data, bandwidth)

# Print all possible arrive times
for cost, time, path in all_arrive_times:
    print(f"Cost: {cost}, Arrival Time: {time}, Path: {path}")

Cost: 50, Arrival Time: 20, Path: [(0, 2)]
Cost: 60, Arrival Time: 15, Path: [(0, 2)]
Cost: 70, Arrival Time: 10, Path: [(0, 2)]
Cost: 80, Arrival Time: 40, Path: [(0, 1), (1, 2)]
Cost: 90, Arrival Time: 35, Path: [(0, 1), (1, 2)]
Cost: 100, Arrival Time: 30, Path: [(0, 1), (1, 2)]
Cost: 120, Arrival Time: 30, Path: [(0, 1), (1, 2)]


In [6]:
import numpy as np
from scipy.stats import gaussian_kde

def calculate_kde_weight(xi, xj, density_estimator):
    return density_estimator.pdf((xi + xj) / 2) * np.linalg.norm(xi - xj)

def find_all_arrive_times(adjacency_matrix, cost_tables, start_vertex, end_vertex, max_cost, density_estimator):
  
  # Initialize DP table
  num_vertices = len(adjacency_matrix)
  dp = [[float('inf')] * (max_cost + 1) for _ in range(num_vertices)]
  path = [[[] for _ in range(max_cost + 1)] for _ in range(num_vertices)]

  # Initialize starting vertex
  dp[start_vertex][0] = 0
  
  # Function to calculate KDE weight

  # Iterate through vertices
  for v in range(num_vertices):
    for cost in range(max_cost + 1):
      if dp[v][cost] == float('inf'): # Skip if vertex v is not reachable
        continue

      # Iterate through neighbors of vertex v
      for u in range(num_vertices):
        if v != u and adjacency_matrix[v][u] == 1 and v != end_vertex and (v, u) in cost_tables:  # Check if edge exists;  check (v, u) in cost_tables: data may be missing
          for edge_cost, edge_time in cost_tables[(v, u)]:
            new_cost = cost + edge_cost
            new_time = dp[v][cost] + edge_time

            # Check if the new cost is within budget
            if new_cost <= max_cost:
              # Update the DP table and path if the new time is shorter
              # Update the DP table and path with KDE weights
              kde_weight = calculate_kde_weight(synthetic_data[v], synthetic_data[u], density_estimator)
              if new_time < dp[u][new_cost]:
                dp[u][new_cost] = new_time
                path[u][new_cost] = path[v][cost] + [(v, u)]
                cost_tables[(v, u)] = [(kde_weight, edge_time)]
              elif new_time == dp[u][new_cost]:
                # If arrival time is the same, choose the shorter path
                if len(path[v][cost]) + 1 < len(path[u][new_cost]):
                  path[u][new_cost] = path[v][cost] + [(v, u)]

  # Get all possible arrive times
  all_arrive_times = []
  for cost in range(max_cost + 1):
    if dp[end_vertex][cost] != float('inf'):
      all_arrive_times.append((cost, dp[end_vertex][cost], path[end_vertex][cost]))

  return all_arrive_times


In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assuming you have a DataFrame 'df' with your dataset
# Replace this with your actual dataset loading code
# For example, df = pd.read_csv('your_dataset.csv')
# Make sure to preprocess your data if needed (e.g., handle categorical variables)

# Sample DataFrame creation (replace this with your actual dataset loading code)
data = {
    'age': [25, 30, 35, 40, 45],
    'workclass': ['Private', 'Government', 'Private', 'Government', 'Private'],
    'fnlwgt': [2000, 3000, 2500, 3500, 4000],
    # Add other features as needed
}

df = pd.DataFrame(data)

# Extract features for KNN
features = df[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]

# Choose the data point for which you want to find neighbors
target_data_point = [30, 2800, 12, 1000, 0, 40]  # Replace with the actual values

# Create a KNN model
knn_model = NearestNeighbors(n_neighbors=5, algorithm='auto')  # Adjust n_neighbors as needed

# Fit the model with your dataset
knn_model.fit(features)

# Find the indices of the nearest neighbors
distances, indices = knn_model.kneighbors([target_data_point])

# Extract neighbor data points
neighbor_indices = indices[0]
neighbors = df.iloc[neighbor_indices]

# Print the neighbors
print("Nearest neighbors:")
print(neighbors)


ModuleNotFoundError: No module named 'pandas'